In [1]:
 import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings

sys.path.append(os.path.abspath(os.path.join('../scripts')))

from clean_tellco_dataframe import Tellco
from plot_dataframe import Plotter

plot = Plotter()
cleaner = Tellco()

In [2]:
warnings.filterwarnings('ignore')

In [25]:
tellco_df = pd.read_csv("../data/clean_data.csv")

In [26]:
tellco_exprience_df = tellco_df[['MSISDN/Number','Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Handset Type']]
tellco_exprience_df

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type
0,33664962239,42.0,5.0,23.0,44.0,568730.0,20949.5,Samsung Galaxy A5 Sm-A520F
1,33681854413,65.0,5.0,16.0,26.0,568730.0,20949.5,Samsung Galaxy J5 (Sm-J530)
2,33760627129,45.0,5.0,6.0,9.0,568730.0,20949.5,Samsung Galaxy A8 (2018)
3,33699795932,45.0,5.0,6.0,9.0,568730.0,20949.5,Samsung Sm-G390F
4,33665368271,102.0,5.0,22.0,38.0,568730.0,20949.5,Huawei Mate 20 Lite
...,...,...,...,...,...,...,...,...
138066,33645655643,45.0,5.0,55.0,54.0,568730.0,20949.5,Samsung Galaxy J5 (Sm-J530)
138067,33650688697,32.0,0.0,52.0,65.0,568730.0,20949.5,Apple iPhone 8 Plus (A1897)
138068,33663449963,27.0,2.0,23.0,54.0,568730.0,20949.5,Apple iPhone Se (A1723)
138069,33621890103,43.0,6.0,43.0,47.0,568730.0,20949.5,Apple iPhone Xs (A2097)


In [27]:
tellco_exprience_df['Total Avg RTT (ms)'] = tellco_exprience_df['Avg RTT DL (ms)'] + tellco_exprience_df['Avg RTT UL (ms)']
tellco_exprience_df['Total Avg Bearer TP (kbps)'] = tellco_exprience_df['Avg Bearer TP DL (kbps)'] + tellco_exprience_df['Avg Bearer TP UL (kbps)']
tellco_exprience_df['Total TCP Retrans. Vol (Bytes)'] = tellco_exprience_df['TCP DL Retrans. Vol (Bytes)'] + tellco_exprience_df['TCP UL Retrans. Vol (Bytes)']
tellco_exprience_df.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33664962239,42.0,5.0,23.0,44.0,568730.0,20949.5,Samsung Galaxy A5 Sm-A520F,47.0,67.0,589679.5
1,33681854413,65.0,5.0,16.0,26.0,568730.0,20949.5,Samsung Galaxy J5 (Sm-J530),70.0,42.0,589679.5
2,33760627129,45.0,5.0,6.0,9.0,568730.0,20949.5,Samsung Galaxy A8 (2018),50.0,15.0,589679.5
3,33699795932,45.0,5.0,6.0,9.0,568730.0,20949.5,Samsung Sm-G390F,50.0,15.0,589679.5
4,33665368271,102.0,5.0,22.0,38.0,568730.0,20949.5,Huawei Mate 20 Lite,107.0,60.0,589679.5


In [28]:
tellco_exprience_df = tellco_exprience_df[['MSISDN/Number','Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)','Handset Type']]
tellco_exprience_df.head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33664962239,47.0,67.0,589679.5,Samsung Galaxy A5 Sm-A520F
1,33681854413,70.0,42.0,589679.5,Samsung Galaxy J5 (Sm-J530)
2,33760627129,50.0,15.0,589679.5,Samsung Galaxy A8 (2018)
3,33699795932,50.0,15.0,589679.5,Samsung Sm-G390F
4,33665368271,107.0,60.0,589679.5,Huawei Mate 20 Lite


In [36]:
tellco_exprience_df = tellco_exprience_df.copy()
tellco_exprience_df = tellco_exprience_df.groupby(
    'MSISDN/Number').agg({'Total Avg RTT (ms)': 'sum', 'Total Avg Bearer TP (kbps)': 'sum', 'Total TCP Retrans. Vol (Bytes)': 'sum','Handset Type': lambda x: x.mode() }) #' '.join(x)
tellco_exprience_df

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33601001722,46.0,76.0,589679.5,Huawei P20 Lite Huawei Nova 3E
33601001754,31.0,99.0,589679.5,Apple iPhone 7 (A1778)
33601007832,84.0,248.0,589679.5,Apple iPhone 5S (A1457)
33601008617,119.0,43342.0,1179359.0,Apple iPhone Se (A1723)
33601010682,74.0,7908.0,1179359.0,Samsung Galaxy A8 (2018)
...,...,...,...,...
33789960306,50.0,106.0,589679.5,Samsung Galaxy S8 Plus (Sm-G955F)
33789967113,53.0,67.0,589679.5,Apple iPhone 6S (A1688)
33789996170,52.0,10365.0,589679.5,Huawei Honor 9 Lite


In [41]:
tp = tellco_exprience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False)
tp.head(10)['Total Avg Bearer TP (kbps)']

60307     55060.0
126504    55060.0
71940     55060.0
131719    55060.0
12134     55060.0
126499    55060.0
71949     55060.0
12144     55060.0
71952     55060.0
126497    55060.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [42]:
tp.tail(10)['Total Avg Bearer TP (kbps)']

84742     0.0
8737      0.0
61769     0.0
123025    0.0
13773     0.0
565       0.0
12998     0.0
73824     0.0
19492     0.0
2321      0.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [44]:
tellco_exprience_df['Total Avg Bearer TP (kbps)'].value_counts().head(10)

55060.0    6646
63.0       3800
15.0       2720
97.0       1799
90.0       1773
98.0       1673
96.0       1551
89.0       1475
99.0       1450
93.0       1446
Name: Total Avg Bearer TP (kbps), dtype: int64

In [45]:
br_tp = tellco_exprience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False)
br_tp

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
60307,33761191059,94.5,55060.0,589679.5,Huawei B528S-23A
126504,33683931728,65.0,55060.0,589679.5,Apple iPhone 7 Plus (A1784)
71940,33667663042,63.0,55060.0,589679.5,Huawei B528S-23A
131719,33667076990,108.5,55060.0,589679.5,Huawei B528S-23A
12134,33635211255,44.0,55060.0,589679.5,Apple iPhone X (A1901)
...,...,...,...,...,...
565,33658211925,50.0,0.0,589679.5,Wct Uno
12998,33659471143,50.0,0.0,589679.5,Huawei P20 Pro
73824,33662540046,50.0,0.0,589679.5,Huawei B528S-23A
19492,33604131582,50.0,0.0,589679.5,Lenovo Moto G (3Rd Gen) Ca12459445


In [46]:
br_tp.head(10)['Total Avg Bearer TP (kbps)']

60307     55060.0
126504    55060.0
71940     55060.0
131719    55060.0
12134     55060.0
126499    55060.0
71949     55060.0
12144     55060.0
71952     55060.0
126497    55060.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [47]:
br_tp.tail(10)['Total Avg Bearer TP (kbps)']

84742     0.0
8737      0.0
61769     0.0
123025    0.0
13773     0.0
565       0.0
12998     0.0
73824     0.0
19492     0.0
2321      0.0
Name: Total Avg Bearer TP (kbps), dtype: float64

In [48]:
tellco_exprience_df['Total Avg Bearer TP (kbps)'].value_counts().head(10)

55060.0    6646
63.0       3800
15.0       2720
97.0       1799
90.0       1773
98.0       1673
96.0       1551
89.0       1475
99.0       1450
93.0       1446
Name: Total Avg Bearer TP (kbps), dtype: int64

In [50]:
tcp_retransmitted = tellco_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False)
tcp_retransmitted

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33664962239,47.0,67.0,589679.5,Samsung Galaxy A5 Sm-A520F
92043,33659805846,16.0,90.0,589679.5,Apple iPhone Xs (A2097)
92057,33760979434,62.0,23845.0,589679.5,Huawei Mate 10 Lite Huawei Nova 2I
92056,33781756201,48.0,68.0,589679.5,Apple iPhone 7 (A1778)
92055,33664211332,68.0,69.0,589679.5,Apple iPhone 6S (A1688)
...,...,...,...,...,...
46018,33666216760,50.0,85.0,589679.5,Samsung Galaxy A5 Sm-A520F
46017,33668983153,28.0,110.0,589679.5,Apple iPhone 8 (A1905)
46016,33664754536,50.0,82.0,589679.5,Samsung Galaxy S7 (Sm-G930X)
46015,33670459917,50.0,109.0,589679.5,Samsung Galaxy S7 (Sm-G930X)


In [51]:
tcp_retransmitted.head(10)['Total TCP Retrans. Vol (Bytes)']

0        589679.5
92043    589679.5
92057    589679.5
92056    589679.5
92055    589679.5
92054    589679.5
92053    589679.5
92052    589679.5
92051    589679.5
92050    589679.5
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [52]:
tcp_retransmitted.tail(10)['Total TCP Retrans. Vol (Bytes)']

46023     589679.5
46022     589679.5
46021     589679.5
46020     589679.5
46019     589679.5
46018     589679.5
46017     589679.5
46016     589679.5
46015     589679.5
138070    589679.5
Name: Total TCP Retrans. Vol (Bytes), dtype: float64

In [53]:
# tellco_exprience_df['Total TCP Retrans. Vol (Bytes)'].value_counts().head(10)

589679.5    138071
Name: Total TCP Retrans. Vol (Bytes), dtype: int64

In [57]:
handset_type_exprience_df = tellco_exprience_df.groupby('Handset Type').mean()
handset_type_exprience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False).head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
Handset Type,,,,
Samsung Galaxy S9+,3.364948e+10,39.0,55060.0,589679.5
Oppo Cph1723,3.365881e+10,58.5,55060.0,589679.5
Zyxel Communicat. Sbg3600,3.366811e+10,117.5,55060.0,589679.5
Huawei E8372,3.365883e+10,100.5,55060.0,589679.5
Ovvi-Cellphone T. Echo,3.369858e+10,59.0,55060.0,589679.5


In [58]:
handset_type_exprience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False).head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
Handset Type,,,,
A-Link Telecom I. Cubot A5,3.377002e+10,42.00,23510.000000,589679.5
Samsung Galaxy S3 (Gt-I9305),3.368538e+10,70.25,7230.833333,589679.5
Samsung Galaxy S4 Active Gt-I9295,3.366205e+10,41.00,40242.000000,589679.5
Samsung Galaxy S4 (Sgh-I337M),3.365801e+10,109.50,182.000000,589679.5
Samsung Galaxy S4 (Gt-I9515),3.369944e+10,50.25,17175.000000,589679.5


In [ ]:
tellco_exprience_df.drop(['Handset Type'], axis=1, inplace=True)

In [60]:

scale_data = StandardScaler().fit_transform(tellco_exprience_df)
scale_data

array([[-0.00270178, -0.46659147, -0.65946518,  0.        ],
       [-0.00268315,  0.2980162 , -0.66079607,  0.        ],
       [-0.00259629, -0.36686004, -0.66223343,  0.        ],
       ...,
       [-0.00270345, -1.06498009, -0.65893283,  0.        ],
       [-0.00274928, -0.40010385, -0.65824077,  0.        ],
       [-0.00275178, -0.63281053, -0.65925224,  0.        ]])

In [61]:
normalized_data = normalize(scale_data)
normalized_data

array([[-0.00334444, -0.5775774 , -0.81632908,  0.        ],
       [-0.00370144,  0.41111655, -0.91157527,  0.        ],
       [-0.00342942, -0.48458249, -0.87473884,  0.        ],
       ...,
       [-0.0021587 , -0.85038485, -0.52615678,  0.        ],
       [-0.00356907, -0.51940888, -0.85451839,  0.        ],
       [-0.00301129, -0.6924879 , -0.7214232 ,  0.        ]])

In [62]:
kmeans = KMeans(n_clusters = 3, random_state = 42)
y_kmeans = kmeans.fit_predict(normalized_data)
X = np.array(normalized_data)
y_kmeans

array([0, 1, 0, ..., 0, 0, 0])

In [67]:
clustered_tellco_exprience_df = tellco_exprience_df.copy()
clustered_tellco_exprience_df.insert(0, 'Cluster', y_kmeans)
clustered_tellco_exprience_df

,Cluster,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,0,33664962239,47.0,67.0,589679.5
1,1,33681854413,70.0,42.0,589679.5
2,0,33760627129,50.0,15.0,589679.5
3,0,33699795932,50.0,15.0,589679.5
4,1,33665368271,107.0,60.0,589679.5
...,...,...,...,...,...
138066,0,33645655643,50.0,109.0,589679.5
138067,0,33650688697,32.0,117.0,589679.5
138068,0,33663449963,29.0,77.0,589679.5
138069,0,33621890103,49.0,90.0,589679.5


In [68]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==0][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,78718.000000,78718.000000,78718.0
mean,42.249015,799.404698,589679.5
std,11.531014,2535.257585,0.0
min,0.000000,0.000000,589679.5
25%,32.000000,79.000000,589679.5
50%,46.000000,98.000000,589679.5
75%,50.000000,121.000000,589679.5
max,67.000000,22811.000000,589679.5


In [69]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==1][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,27348.000000,27348.000000,27348.0
mean,103.533695,10889.878126,589679.5
std,21.187050,11729.917892,0.0
min,50.000000,0.000000,589679.5
25%,82.000000,132.000000,589679.5
50%,109.500000,6802.500000,589679.5
75%,119.500000,19556.000000,589679.5
max,132.000000,44449.000000,589679.5


In [70]:
clustered_tellco_exprience_df[clustered_tellco_exprience_df["Cluster"]==2][["Total Avg RTT (ms)","Total Avg Bearer TP (kbps)","Total TCP Retrans. Vol (Bytes)"]].describe()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
count,32005.000000,32005.000000,32005.0
mean,70.927246,42458.664052,589679.5
std,26.149888,12345.752460,0.0
min,19.000000,12475.000000,589679.5
25%,50.000000,32264.000000,589679.5
50%,67.000000,45629.000000,589679.5
75%,87.500000,54271.000000,589679.5
max,132.000000,55060.000000,589679.5
